In [19]:
import pandas as pd
import sqlite3
df = pd.read_excel('/content/cleaned_data.xlsx')
conn = sqlite3.connect(':memory:')
df.to_sql('my_table', conn, index=False)
query = 'SELECT * FROM my_table'
result_df = pd.read_sql(query, conn)
print(result_df.head())

   Unnamed: 0  Ind_ID   Test_date  Cough_symptoms  Fever  Sore_throat  \
0           0       1  11-03-2020               1      0            1   
1           1       2  11-03-2020               0      1            0   
2           2       3  11-03-2020               0      1            0   
3           3       4  11-03-2020               1      0            0   
4           4       5  11-03-2020               1      0            0   

   Shortness_of_breath  Headache    Corona Age_60_above     Sex  \
0                    0         0  negative           No  female   
1                    0         0  positive           No  female   
2                    0         0  positive           No  female   
3                    0         0  negative           No  female   
4                    0         0  negative           No  female   

            Known_contact  
0                  Abroad  
1                  Abroad  
2                  Abroad  
3                  Abroad  
4  Contact with co

1.Find the number of corona patients who faced shortness of breath?

In [41]:
query = "SELECT COUNT(*) AS num_patients_shortness_of_breath FROM my_table WHERE Corona = 'positive' AND Shortness_of_breath = TRUE"
result_df = pd.read_sql(query, conn)
print(result_df.head())

   num_patients_shortness_of_breath
0                               415


2.Find the number of negative corona patients who have fever and sore_throat?

In [40]:

query = "SELECT COUNT(*) AS num_negative_patients_fever_sore_throat FROM my_table WHERE Corona = 'negative' AND Fever = TRUE AND Sore_throat = TRUE"
result_df = pd.read_sql(query, conn)
print(result_df.head())

   num_negative_patients_fever_sore_throat
0                                       88


3.Group the data by month and rank the number of positive cases?

In [31]:
query = """SELECT strftime('%m', Test_date) AS month_,
       COUNT(*) AS num_positive_cases,
       DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS rank_
FROM my_table
WHERE Corona = 'positive'
GROUP BY month_;"""

result_df = pd.read_sql(query, conn)
print(result_df.head())

  month_  num_positive_cases  rank_
0   None                2375      1


4.Find the female negative corona patients who faced cough and headache?

In [39]:
query = """ SELECT COUNT(*) AS num_female_negative_patients_cough_headache FROM my_table
WHERE Corona = 'negative' AND Sex = 'female' AND Cough_symptoms = TRUE AND Headache = TRUE;
"""
result_df = pd.read_sql(query, conn)
print(result_df.head())

   num_female_negative_patients_cough_headache
0                                           38


5.How many elderly corona patients have faced breathing problems?

In [38]:
query = """ SELECT COUNT(*) AS num_elderly_patients_breathing_problems
FROM my_table
WHERE Corona = 'positive' AND Age_60_above = TRUE AND Shortness_of_breath = TRUE ;"""
result_df = pd.read_sql(query, conn)
print(result_df.head())

   num_elderly_patients_breathing_problems
0                                        0


6.. Which three symptoms were more common among COVID positive patients?

In [43]:
query = """ SELECT Cough_symptoms, Fever, Sore_throat, COUNT(*) AS num_cases
FROM my_table
WHERE Corona = 'positive'
GROUP BY Cough_symptoms, Fever, Sore_throat
ORDER BY num_cases DESC
LIMIT 3;"""
result_df = pd.read_sql(query, conn)
print(result_df.head())

   Cough_symptoms  Fever  Sore_throat  num_cases
0               1      1            0        658
1               0      0            0        633
2               1      0            0        356


7. Which symptom was less common among COVID negative people?

In [45]:
query = """ SELECT CASE
    WHEN Cough_symptoms = 'TRUE' THEN 'Cough'
    WHEN Fever = 'TRUE' THEN 'Fever'
    WHEN Sore_throat = 'TRUE' THEN 'Sore throat'
    WHEN Shortness_of_breath = 'TRUE' THEN 'Shortness of breath'
    WHEN Headache = 'TRUE' THEN 'Headache'
    ELSE 'No symptoms'
    END AS symptom, COUNT(*) AS num_negative_cases
FROM my_table
WHERE Corona = 'negative'
GROUP BY symptom
ORDER BY num_negative_cases
LIMIT 5;"""
result_df = pd.read_sql(query, conn)
print(result_df.head())


       symptom  num_negative_cases
0  No symptoms               26681


8. What are the most common symptoms among COVID positive males whose known contact was abroad


In [47]:
query = """ SELECT Cough_symptoms, Fever, Sore_throat, Headache, Shortness_of_breath, COUNT(*) AS num_cases
FROM my_table
WHERE Corona = 'positive' AND Sex = 'male' AND Known_contact = 'Abroad'
GROUP BY Cough_symptoms, Fever, Sore_throat, Headache, Shortness_of_breath
ORDER BY num_cases DESC
LIMIT 1;
"""
result_df = pd.read_sql(query, conn)
print(result_df.head())

   Cough_symptoms  Fever  Sore_throat  Headache  Shortness_of_breath  \
0               1      1            0         0                    0   

   num_cases  
0        104  
